In [1]:
from os import listdir
import pandas as pd
from collections import Counter
from Bio import SeqIO
from Bio import Seq

vcfs="/home/lshas17/converted_vcfs/"
new_vcfs="/home/lshas17/converted_vcfs_plus_B/"
metadata_file="/home/lshas17/HandyAmpliconTool/test_data/TGC_plus_B.csv"
rep_samples_file="/home/lshas17/HandyAmpliconTool/test_data/inputs/representative_samples.tsv"
gff_file="/home/lshas17/HandyAmpliconTool/test_data/inputs/GCA_000195995.1_for_VCFs.gff"
rep_samples_vcf_file="/home/lshas17/HandyAmpliconTool/test_data/inputs/Zoe_vcfs_merged.vcf"
fasta_file="/home/lshas17/HandyAmpliconTool/test_data/inputs/GCA_000195995.1_for_VCFs.fasta"

vcf_files=[f for f in listdir(vcfs) if f.split(".")[-1]=="vcf"]
new_vcf_files=[f for f in listdir(new_vcfs) if f.split(".")[-1]=="vcf" and f not in vcf_files]

In [2]:
meta_data=pd.read_csv(metadata_file, sep=",", index_col=0)
if Counter(meta_data.index.duplicated())[True]!=0:
    duplicate_indices=meta_data.index[meta_data.index.duplicated()]
rep_samples=[f.strip() for f in open(rep_samples_file)]
genes_vcf={}
for line in open(gff_file):
    if line[0]!="#":
        values=line.strip().split("\t")
        if values[2]=="gene":
            info_values=values[8].strip().split(";")
            genes_vcf[(int(values[3]), int(values[4]))] = {"ID":info_values[0].replace("ID=",""),"Name":info_values[1].replace("Name=",""), "Orientation":values[6]}
ref_seq=Seq.Seq("")
for record in SeqIO.parse(fasta_file, "fasta"):
    if record.id=="AL513382_143N_pHCM1_120N_pHCM2":
        ref_seq=record.seq

In [3]:
new_gt_positions={}
for line in open("/home/lshas17/HandyAmpliconTool/test_data/outputs/genotype_snps_B.bed"):
    values=line.split("\t")
    sub_values=values[3].split(":")
    alleles=sub_values[1].split("/")
    new_gt_positions[ int(values[2]) ] = {"Ref": alleles[0],"Alt": alleles[1]}


gts_in_genes=[]
valid_positions=[]
gene_unique_snps={}
for position in new_gt_positions.keys():
    for key,value in genes_vcf.items():
        if position in range(key[0], key[1]):
            gts_in_genes.append(position)
            valid_positions+=list(range(key[0], key[1]))
            gene_unique_snps[ key ]=0
            break
valid_positions=set(valid_positions)

In [58]:
lines_to_skip=0
for line in open(rep_samples_vcf_file):
    if line[0:2]=="##":
        lines_to_skip+=1

representative_vcfs = pd.read_csv(rep_samples_vcf_file, sep="\t", skiprows=lines_to_skip-1, header=1)
rep_genes_vcf = representative_vcfs.loc[ [index for index in representative_vcfs.index if representative_vcfs.loc[index,"POS"] in valid_positions] ]

for key in gene_unique_snps:
    gene_unique_snps[key]=0
for position in rep_genes_vcf["POS"]:
    for key in gene_unique_snps.keys():
        if position>=key[0] and position<=key[1]:
            gene_unique_snps[key]+=1

position_impact={}
for position, alleles in new_gt_positions.items():
    position_impact[position]="Other"
    for key, value in genes_vcf.items():
        if position>=key[0] and position<key[1]:
            alt_seq=str(ref_seq)
            alt_seq=alt_seq[0:position-1]+alleles["Alt"]+alt_seq[position:]
            alt_seq=Seq.Seq(alt_seq)
            #check which SNPs are synonimous
            if len(alleles["Ref"])>1 or len(alleles["Alt"])>1:
                position_impact[position]="missense"
            elif genes_vcf[key]["Orientation"]=="+":
                ref_prot=ref_seq[key[0]-1:key[1]].translate()
                alt_prot=alt_seq[key[0]-1:key[1]].translate()
                print([(a,b ) for a,b in zip(ref_prot, alt_prot) if a!=b])
            else:
                ref_prot=ref_seq[key[0]-1:key[1]].reverse_complement().translate()
                alt_prot=alt_seq[key[0]-1:key[1]].reverse_complement().translate()
                print([(a,b ) for a,b in zip(ref_prot, alt_prot) if a!=b])
            position_impact[position]="syn" if ref_prot==alt_prot else "missense"



[('S', 'L')]
[('D', 'N')]
[('G', 'D')]
[('G', 'S')]
[]
[('G', 'S')]
[('L', 'Q')]
[]
[('H', 'Y')]
[]
[('D', 'N')]
[('A', 'E')]
[('T', 'I')]
[('G', 'C')]
[('A', 'P')]
[('M', 'I')]


In [ ]:
for position, values in new_gt_positions.items():
    print(f' {ref_seq[position-1]} at {position} Same as Ref  {values["Ref"]==ref_seq[position-1]} ')

In [49]:
genes_vcf

{(190, 255): {'ID': 'gene-STY_RS00005', 'Name': 'thrL', 'Orientation': '+'},
 (337, 2799): {'ID': 'gene-STY_RS00010', 'Name': 'thrA', 'Orientation': '+'},
 (2801, 3730): {'ID': 'gene-STY_RS00015', 'Name': 'thrB', 'Orientation': '+'},
 (3734, 5020): {'ID': 'gene-STY_RS00020', 'Name': 'thrC', 'Orientation': '+'},
 (5114, 5887): {'ID': 'gene-STY_RS00025', 'Name': 'yaaA', 'Orientation': '-'},
 (5966, 7396): {'ID': 'gene-STY_RS00030',
  'Name': 'STY_RS00030',
  'Orientation': '-'},
 (7665, 8618): {'ID': 'gene-STY_RS00035', 'Name': 'tal', 'Orientation': '+'},
 (8729, 9319): {'ID': 'gene-STY_RS00040', 'Name': 'mog', 'Orientation': '+'},
 (9376, 9942): {'ID': 'gene-STY_RS00045', 'Name': 'satP', 'Orientation': '-'},
 (10092, 10805): {'ID': 'gene-STY_RS00050',
  'Name': 'msyB',
  'Orientation': '-'},
 (10841, 11245): {'ID': 'gene-STY_RS00055',
  'Name': 'STY_RS00055',
  'Orientation': '-'},
 (11594, 13510): {'ID': 'gene-STY_RS00060',
  'Name': 'dnaK',
  'Orientation': '+'},
 (13596, 14735): {'ID

In [55]:
output=pd.DataFrame(columns=["Chr","POS","Ref","Alt","Type","GeneID","GeneName","snps_in_gene","snp_rate"], index=new_gt_positions.keys())
for key, value in gene_unique_snps.items():
    for position, snp_type in new_gt_positions.items():
        if position>=key[0] and position<key[1]:
            output.loc[position,["Chr","POS","Ref","Alt","Type","GeneID","GeneName","snps_in_gene","snp_rate"]] = \
                ["AL513382_143N_pHCM1_120N_pHCM2", position,new_gt_positions[position]["Ref"],new_gt_positions[position]["Alt"],position_impact[position],
                 genes_vcf[key]["ID"], genes_vcf[key]["Name"], value, value/(key[1]-key[0])*1000]
            #print(f'{genes_vcf[key]["Name"]} at {str(key[0])} to {str(key[1])} has {str(value)} or {value/(key[1]-key[0])*1000}, has synonimous SNP at {str(position)}')
            #break
output.to_csv("/home/lshas17/HandyAmpliconTool/test_data/outputs/temp.tsv", sep="\t")

In [35]:
# Double check consistency of identified SNPs vs TGC data
in_old_vcfs={}
missing_in_new={}
for gt_position in gts_in_genes:
    print(gt_position)
    for file in vcf_files:
        with open(f'{vcfs}{file}') as old_file:
            for line in old_file:
                if line[0]!="#" and line!="\n" and int(line.split("\t")[1]) in gts_in_genes:
                    if gt_position not in in_old_vcfs:
                        in_old_vcfs[gt_position]=[]
                    in_old_vcfs[gt_position].append(file)
                    break
    for file in new_vcf_files:
        no_snp=True
        with open(f'{new_vcfs}{file}') as new_file:
            for line in new_file:
                if line[0]!="#" and line!="\n" and int(line.split("\t")[1]) in new_gt_positions:
                    no_snp=False
                    break
            if no_snp:
                if gt_position not in missing_in_new:
                    missing_in_new[gt_position]=[]
                missing_in_new[gt_position].append(file)

print(in_old_vcfs)
print(set([f for val in missing_in_new.values() for f in val]))

592774
4636043
1499665
2380436
4552596
1834775
4643095
2080299
3246383
561718
4616638
4616647
3014988
3789427
1275898
3405819
3499647
